# Code demo 19/05/2021 Flagship

###### 

In [ ]:
pwd

- petit discours d'introduction

In [1]:
# Import des librairies

import os
import pandas as pd
import psycopg2
import csv
import reprlib
import numpy
import re
import json

###### 

-----------------------

### Création de la Base de données

In [2]:
"""
Connexion dans un premier temps à postgres
pour initialiser notre database collection
"""


# initialisation des variables  de connection
utilisateur = "postgres"

mot_passe = os.environ.get('pg_psw')

base_de_donnees = "postgres"



try:
    
    # création de l'objet conn

    conn = psycopg2.connect(dbname= base_de_donnees, user=utilisateur, password=mot_passe, host="localhost", port= 5432)

    conn.set_session(autocommit=True)   # on force

    # initialisation du cursor

    cursor = conn.cursor()

    # cursor.execute permet d'exécuter une commande 'sql'

    cursor.execute("CREATE DATABASE collection4")

    print("La DataBase à été créée avec succès !")

        
except psycopg2.Error as e:
    
    print("Problème lors de la connection à la base de donnée")

    print(e)

   
    
finally:
        
    # ferme le curseur et devient inutilisable

    cursor.close

    # ferme la connexion à database

    conn.close

Problème lors de la connection à la base de donnée
ERREUR:  la base de données « collection4 » existe déjà



######

In [3]:
# Cellule pour nos fonctions de connection

# Fonction qui va nous permettre de créer une Base de données.
def ouvrir_connection(base_de_donnee, utilisateur, mot_passe, host='localhost', port=5432):
    
    try:
        # création de l'objet conn
        conn = psycopg2.connect(dbname= base_de_donnee, user=utilisateur, password = mot_passe, host="localhost", port= 5432)

        # on force la sauvegarde
        conn.set_session(autocommit=True)   
        
        # initialisation du cursor
        cursor = conn.cursor()
        
        # exécution de la requête sql de création de la base de donnée
        #cursor.execute("CREATE DATABASE IF NOT EXISTS movie")

    # gestion des erreurs    
    except psycopg2.Error as e:
        print("Problème lors de la connection à la base de donnée")
        print(e)
        return None
    # dans tous les cas on va clôturer la session
    finally:
        cursor.close
        conn.close
    return conn
   
    # fonction pour supprimer une table
def supprimer_table(conn, sql_suppression_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_suppression_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la suppression de la table")
        print(e)
        return
    cursor.close()
    print("La table a été supprimée avec succès")

# fonction pour créer une table    
def creer_table(conn, sql_creer_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_creer_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la création de la table")
        print(e)
        return
    cursor.close()
    print("La table a été crée avec succès")

# fonction de lecture de la table    
def lire_table(conn, sql_lire_donnee):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_lire_donnee)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la lecture des données")
        print(e)
        return None
    
    print("Les données ont été lues avec succès")
    
    data = []
    for row in cursor:
        data.append(row)

    cursor.close()
    
    return data   

In [4]:
"""cellule qui content différentes requêtes sql
"""

# requète création table d'origine
sql_creer_table1 = """CREATE TABLE IF NOT EXISTS Origine(
   code_Origine SERIAL,
   nom_origine VARCHAR(100),
   PRIMARY KEY(Code_Origine)
);
"""
# requète création table producteur
sql_creer_table2 = """CREATE TABLE IF NOT EXISTS Producteur(
   code_producteur SERIAL,
   nom_producteur VARCHAR(100),
   code_Origine INT,
   PRIMARY KEY(Code_producteur),
   FOREIGN KEY(Code_Origine) REFERENCES Origine(Code_Origine)
);
"""
# requète création table produit
sql_creer_table3 = """CREATE TABLE IF NOT EXISTS Produit(
   code_produit SERIAL,
   nom_produit VARCHAR(250),
   volume INT,
   degre DECIMAL(15,2),
   code_producteur INT,
   PRIMARY KEY(Code_produit),
   FOREIGN KEY(Code_producteur) REFERENCES Producteur(Code_producteur)
);
"""
sql_creer_table4 = """CREATE TABLE IF NOT EXISTS Utilisateur(
   code_utilisateur SERIAL,
   nom VARCHAR(100),
   surnom VARCHAR(100),
   email VARCHAR(150),
   PRIMARY KEY(code_utilisateur)
);
"""
sql_creer_table5 = """CREATE TABLE IF NOT EXISTS Collection(
   Code_collection SERIAL,
   nom VARCHAR(250),
   description VARCHAR(500),
   code_utilisateur INT,
   PRIMARY KEY(Code_collection),
   UNIQUE(code_utilisateur),
   FOREIGN KEY(code_utilisateur) REFERENCES Utilisateur(code_utilisateur)
);

"""

sql_creer_table6 = """CREATE TABLE IF NOT EXISTS Compose(
   Code_produit INT,
   Code_collection INT,
   description VARCHAR(250),
   PRIMARY KEY(Code_produit, Code_collection),
   FOREIGN KEY(Code_produit) REFERENCES Produit(Code_produit),
   FOREIGN KEY(Code_collection) REFERENCES Collection(Code_collection)
);
"""
sql_creer_table7 = """CREATE TABLE role(
   code_role INT,
   nom VARCHAR(50),
   PRIMARY KEY(code_role)
);
"""

sql_creer_table8 = """CREATE TABLE Droit(
   code_droit INT,
   permission VARCHAR(50),
   PRIMARY KEY(code_droit)
);
"""
# requète lecture des données
sql_lire_donnee = """
    SELECT *
    FROM Produit
    LIMIT 10;
"""

In [5]:
# initialisation des variables pour me connecter à ma bdd postgres

base_de_donnee = "collection4"

utilisateur = "postgres"

mot_passe = os.environ.get('pg_psw')


In [10]:
# initialisation de ma connection
conn = ouvrir_connection(base_de_donnee, utilisateur, mot_passe)

In [ ]:
# création table origine
creer_table(conn, sql_creer_table1)

In [ ]:
# création table producteur
creer_table(conn, sql_creer_table2)

In [ ]:
# création table produit
creer_table(conn, sql_creer_table3)

In [11]:
# création table produit
creer_table(conn, sql_creer_table4)

La table a été crée avec succès


In [12]:
# création table produit
creer_table(conn, sql_creer_table5)

La table a été crée avec succès


In [ ]:
# création table produit
creer_table(conn, sql_creer_table6)

###### 

--------------------------

### Nettoyage des données

###### 

- nettoyage fichier excel

In [ ]:
excel_file = 'INVENTAIRE_A.xlsx'
inventaire = pd.read_excel(excel_file, skiprows=2)
inventaire.drop(99,0,inplace=True)
inventaire.drop(0,0,inplace=True)
del inventaire['Unnamed: 5']
inventaire.columns = ['NOM', 'VOLUME', 'DEGRE', 'NOMBRE', 'PRIX', 'MARQUE']
index_with_nan = inventaire.index[inventaire.isnull().any(axis=1)]
inventaire.drop(index_with_nan,0, inplace=True)
inventaire

In [ ]:
inventaire.info

In [ ]:
inventaire.shape

In [ ]:
inventaire.to_excel(r'INVENTAIRE_B.xlsx', index = False, header=True)

###### 

- nettoyage fichier rhum_data.csv

In [ ]:
csv_file = 'rum_data_brut.csv'
liste_rhum = pd.read_csv(csv_file)

liste_rhum.head(275)

In [ ]:
list_unknown = liste_rhum['company'] == 'Unknown'

In [ ]:
filtered_df = liste_rhum[list_unknown]
print(filtered_df.head())

In [ ]:
for x in liste_rhum.itertuples():
    print(x.company) # Imprime la valeur courante de la colonne A de df
    

In [ ]:
# on change les valeurs inconnus par le nom de la bouteille
liste_rhum.loc[liste_rhum['company'] == 'Unknown', 'company'] = liste_rhum['name']

In [ ]:
liste_rhum.to_csv('rum_data_new2.csv', index = False, header=True)

- remplacement des caractères spéciaux

In [ ]:
csv_file = 'rum_data_new2.csv'
liste_rhum = pd.read_csv(csv_file)

liste_rhum.head(275)

In [ ]:
# df.replace([old_value], new_value)
# création liste caractère spéciaux et leurs valeurs de remplacement
dico = {}
liste_car = ['Ã¡', 'Ã»', 'Ã¢', 'Ã©', 'â€™', 'Â°', 'Ã¨', 'Ã³', 'Ã±', '\'', 'Ã£']
liste_remp = ['a', 'u', 'a', 'e', '', '°', 'e', 'o', 'n', '', 'a']
# création du dictionnaire avec une compréhension de dictionnaire
dico = {k: v for k, v in zip(liste_car, liste_remp)}

In [ ]:
# il faut que regex=True 
liste_rhum.replace({'name': dico}, regex=True,inplace=True)

In [ ]:
liste_rhum.replace({'company': dico}, regex=True,inplace=True)

In [ ]:
liste_rhum.to_csv('rum_data_new3.csv', index = False, header=True)

###### 

- nettoyage fichier json

In [ ]:
from collections import defaultdict
import pandas as pd
import re
import json

origines = []
degres = []
volumes = []
noms = []
producteurs = []
marques = []
liste_json = []

with open('test.json', 'r') as f:
    data = json.load(f)

for rum in data:
    for k, v in rum.items():
        v = v.replace(' cl', 'cl')
        liste_json.append(v)
        
for element in liste_json:
    
    # Origines
    pays = element.split()[-1]
    origines.append(pays)
    if pays:
        element = element.replace(pays, "")
        
    # Producteurs  
    marque_pattern = re.compile(r"[a-zA-Zèé]+[&? [a-zA-Zèé0-9']+ Rhum|[a-zA-Zèé']+[ ]? Rhum|[a-zA-Zèé']+ [Liqueur|Punch|Coffret]+")
    marque = marque_pattern.findall(element)
#     marque = marque.replace("Rhum", "")
    if marque:
        marques.append(marque[0])
    else:
        marques.append("XXXXXXXX")

    # Degres    
    degre_pattern = re.compile(r'\d\d°|\d\d,\d\d?°|\d\d\.\d\d?°')
    degre = degre_pattern.findall(element)
    if degre:
        element = element.replace(degre[0], "")
        degres.append(degre[0])
    else:
        degres.append("")
        
    # Volumes
    volume_pattern = re.compile(r'\d?\dcl|\dL')
    volume = volume_pattern.findall(element)
    if volume:
        element = element.replace(volume[0], "")
        volumes.append(volume[0])
    else:
        volumes.append('')
        
    # Noms rhums    
    noms.append(element) 

# nettoyage Producteurs   
for producteur in marques:
    producteur = re.sub('Rhum|Liqueur|Punch|Coffret', '', producteur)
    producteurs.append(producteur)
    


In [ ]:
import pandas as pd

df1 = pd.DataFrame()
df1['noms'] = noms
df1['producteurs'] = producteurs
df1['origines'] = origines
df1['volumes'] = volumes
df1['degres'] = degres
df1.head(80)

In [ ]:
df1.to_csv('test_json_new.csv', index = False, header=True)

In [ ]:
df1 = pd.read_csv('test_json_new.csv')

In [ ]:
df1['producteurs'] = df1['producteurs'].replace('XXXXXXXX', 'Bumbu') 
df1['producteurs'] = df1['producteurs'].replace('JM', 'Rhum JM') 
df1['producteurs'] = df1['producteurs'].replace('James C', 'Saint James') 
df1['producteurs'] = df1['producteurs'].replace('Depaz Doré', 'Depaz') 
df1['producteurs'] = df1['producteurs'].replace('pour rhu', 'Quai Sud') 
df1['producteurs'] = df1['producteurs'].replace(['Maison L', 'Maison La Mauny'], 'La Mauny') 
df1['origines'] = df1['origines'].replace('70cl', 'Guadeloupe') 
df1.loc[df1['producteurs'] == 'Maison La Mauny', 'producteurs'] = 'La Mauny'
df1.loc[df1['producteurs'] == 'Depaz Doré', 'producteurs'] = 'Depaz'
df1.head(241)

In [ ]:
df1.to_csv('test_json_work.csv', index = False, header=True)

In [ ]:
df1 = pd.read_csv('test_json_work.csv')

In [ ]:
dico = {}
liste_car = ['XXXXXXXX', 'JM', 'Depaz Doré', 'Maison La Mauny', 'Maison L', '\'', 
             'de Cori', 'Longueteau  Vieux Cuvée Confrérie du','Bielle  Vieux Millésime 2009 Cuvée Confrérie du', 
             'Discovery', 'Ambré Brun', 'Crème de', 'Favorite  Vieux coeur de', 'VSOP', '4 tubes de', 'Blanc de', 
             'Latitudes', 'pour rhu', 'James C', 'Depaz Doré', 'Blanc Ice', 'Favorite  Vieux Coeur de',
             'Favorite', 'Bally','é','è']
liste_remp = ['Bumbu', 'Rhum JM', 'Depaz', 'La Mauny', 'La Mauny', '', 
              'Saveur de Coriandre', 'Confrérie du Rhum','Confrérie du Rhum', 'Rhum', '', 
              '', 'Favorite', '', '', '', 'Grand Fond Gallion', 'Quai Sud', 'Saint James', 'Depaz', '',
              'La Favorite', 'La Favorite', 'J. Bally','e', 'e']
# création du dictionnaire avec une compréhension de dictionnaire
dico = {k: v for k, v in zip(liste_car, liste_remp)}

In [ ]:
dico2 = {}
liste_car = ['Galante', '46,2°', 'Trinidad', '20cl', 'cocktail', 'ï']
liste_remp = ['Marie Galante', 'Ecosse', 'Trinite et Tobago', 'Martinique', 'France', 'i']
# création du dictionnaire avec une compréhension de dictionnaire
dico2 = {k: v for k, v in zip(liste_car, liste_remp)}

In [ ]:
# il faut que regex=True 
df1.replace({'producteurs': dico}, regex=True,inplace=True)
df1.replace({'origines': dico2}, regex=True,inplace=True)


In [ ]:
df1.to_csv('test_json_final.csv', index = False, header=True)

###### 

- nettoyage new data

In [ ]:
csv_file = 'rhumfullinfo_wikirum.csv'
new_data = pd.read_csv(csv_file)

new_data.head()

In [ ]:
new_data.isnull().sum()
index_with_nan = new_data.index[new_data.isnull().any(axis=1)]
new_data.drop(index_with_nan,0, inplace=True)

In [ ]:
new_data.columns = ['nom', 'producteur', 'origine', 'volume', 'degre']
new_data['producteur'] = new_data['producteur'].str.replace('*','', regex=True)
new_data['nom'] = new_data['nom'].str.replace('\'','', regex=True)
new_data['producteur'] = new_data['producteur'].str.replace('\'','', regex=True)
new_data['degre'] = new_data['degre'].astype(float)
new_data['volume'] = new_data['volume'].astype(int)
new_data['volume'] = new_data['volume'] / 10

In [ ]:
new_data.to_csv('new_data_wiki.csv', index = False, header=True)

-----------------

### Insertion des données

- origine

In [ ]:
# Prod
# liste des origines pour insertions

import csv
# je crée une liste regroupant ma liste de pays
liste = []
with open('rum_data_new3.csv', newline='', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        # gestion des doublons
        if row[2] not in liste:
            liste.append(row[2])
liste = liste[1:]
print(liste)

In [ ]:
# connexion à la bdd collection
conn = psycopg2.connect(
        user = "postgres",
        password = os.environ.get("pg_psw"),
        host = "localhost",
        port = "5432",
        database = "collection4"
)
cur = conn.cursor()

for pays in liste:
    cur.execute("SELECT origine.nom FROM origine WHERE origine.nom = (%s) ", (pays,))
    verif = cur.fetchone()
    if not verif:
        cur.execute("INSERT INTO origine (nom) VALUES(%s)", (pays,))

conn.commit()
conn.close()

###### 

- insertion producteur

In [ ]:
# prod
# je crée une liste regroupant les pays et les producteurs
liste_co = []
with open('rum_data_new3.csv', newline='', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        if row[1:3] not in liste_co:
            liste_co.append((row[1:3]))
liste_co = liste_co[1:]
print(liste_co[0:10])

In [ ]:
# prod
# fonction d'insertion dans la table producteur
conn = psycopg2.connect(
        user = "postgres",
        password = os.environ.get("pg_psw"),
        host = "localhost",
        port = "5432",
        database = "collection4"
)
cur = conn.cursor()
for producteur, pays in liste_co:
    #print(producteur)
    #print(pays)
    cur.execute(f"""SELECT origine.code_origine 
        FROM origine
        WHERE origine.nom = '{pays}';
        """)
    
    origine = cur.fetchone()[0]
    #print(origine)
    #pays = str(pro[0])
    #print(pays)

    cur.execute(f"INSERT INTO producteur (nom, code_origine) VALUES ('{producteur}', {origine})")
    
cur.close()
conn.commit()
conn.close()

###### 

- insertion produit

In [ ]:
# insertion fichier excel
excel_file = 'INVENTAIRE_B.xlsx'
inventaire = pd.read_excel(excel_file)
inventaire.head()

In [ ]:
inventaire_produit =  inventaire[['NOM', 'VOLUME', 'DEGRE', 'MARQUE']]

In [ ]:
liste_inventaire = [list(x) for x in inventaire_produit.to_numpy()]
print(liste_inventaire[0:10])

In [ ]:
nom = liste_inventaire[0][0]
volume = liste_inventaire[0][1]
degre = liste_inventaire[0][2]
marque = liste_inventaire[0][3]

print(nom)
print(volume)
print(degre)
print(marque)

In [ ]:
conn = psycopg2.connect(
        user = "postgres",
        password = os.environ.get("pg_psw"),
        host = "localhost",
        port = "5432",
        database = "collection4"
)
cur = conn.cursor()
for nom, volume, degre, marque in liste_inventaire:
    print(marque)
    
    cur.execute(f"""SELECT producteur.code_producteur 
        FROM producteur
        WHERE producteur.nom = '{marque}';
        """)
    
    code = cur.fetchone()[0]
    print(code)
    
    cur.execute(f"INSERT INTO produit (nom, volume, degre, code_producteur) VALUES ('{nom}', {volume}, {degre}, {code})")
print(type(cur))   
cur.close()
conn.commit()
conn.close()

In [ ]:
# insertion fichuer csv
# prod
# je crée une liste regroupant les pays et les producteurs
liste_groupe = []
with open('rum_data_new3.csv', newline='', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        liste_groupe.append((row[0:2]))
liste_groupe = liste_groupe[1:]
print(liste_groupe[0:10])

In [ ]:
conn = psycopg2.connect(
        user = "postgres",
        password = os.environ.get("pg_psw"),
        host = "localhost",
        port = "5432",
        database = "collection4"
)
cur = conn.cursor()
for nom, marque in liste_groupe:
    #print(producteur)
    #print(pays)
    cur.execute(f"""SELECT producteur.code_producteur 
        FROM producteur
        WHERE producteur.nom = '{marque}';
        """)
    code = cur.fetchone()[0]
    
    cur.execute(f"INSERT INTO produit (nom, code_producteur) VALUES ('{nom}', {code})")
print(type(cur))   
cur.close()
conn.commit()
conn.close()